## 필요 라이브러리 호출

In [1]:
## 사용 라이브러리 호출
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote
from sklearn.preprocessing import MinMaxScaler

## 모델 사용 라이브러리 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import trange
from sklearn.metrics import mean_squared_error, r2_score

## 모델 학습 결과 경로 설정 
import os
os.makedirs('./result', exist_ok=True)

## Cuda 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 랜덤 시드 설정
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# 시드 설정 
seed_val = 77
set_seed(seed_val)

cuda


## 사용 데이터 컬럼 선택

* tag name 이 많은 경우 tag name을 지정하는 것에 있어서 변수 설정이 다소 유연해짐
* tag name 은 순서대로 불러와짐 

In [2]:
# tag name 출력 함수 
def show_column(URL):
    
    # Tag name 데이터 로드
    df = pd.read_csv(URL)
    
    # List 형식으로 변환
    df = df.values.reshape(-1)
    
    return df.tolist()

In [3]:
## tag name 출력 파라미터 설정
table = 'wind_elec_gen'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get_tag_names.tql?table={table}'

## tag name list 생성 
name = show_column(NAME_URL)

In [4]:
name

['h_6hrain',
 'h_6hsnow',
 'h_humidity',
 'h_rainprobability',
 'h_raintype',
 'h_seawave',
 'h_skystatus',
 'h_temperature',
 'h_winddirection',
 'h_windspeed',
 'hk1_1',
 'hk1_2',
 'hk1_3',
 'hk2_1',
 'hk2_2',
 'hk2_3',
 'hk2_4',
 'hk2_5',
 's_6hrain',
 's_6hsnow',
 's_humidity',
 's_rainprobability',
 's_raintype',
 's_seawave',
 's_skystatus',
 's_temperature',
 's_winddirection',
 's_windspeed',
 'ss_1',
 'ss_10',
 'ss_2',
 'ss_3',
 'ss_4',
 'ss_5',
 'ss_6',
 'ss_7',
 'ss_8',
 'ss_9']

## TAG Name format 변환 

* 위의 과정에서 Wind Elec Gen dataset의 모든 Tag Name 을 확인후 사용할 컬럼만 뽑아서 입력할 파라미터 형태로 변환
* 한경 제 1 발전소 관련 Tag Name 사용

In [5]:
# 원하는 tag name 설정
# 여기서 tag name 은 컬럼을 의미
tags = name[:13]

# 리스트의 각 항목을 작은따옴표로 감싸고, 쉼표로 구분
tags_ = ",".join(f"'{tag}'" for tag in tags)

# 사용 tag name 확인
print(tags_)

# 해당 값을 모델의 input shape로 설정 
print(len(tags))

'h_6hrain','h_6hsnow','h_humidity','h_rainprobability','h_raintype','h_seawave','h_skystatus','h_temperature','h_winddirection','h_windspeed','hk1_1','hk1_2','hk1_3'
13


## Wind Elec Gen Dataset 로드

* 한경 제 1 발전소 관련 Tag Name 들을 사용하여 데이터 로드

In [6]:
# 데이터 로드 파라미터 설정

# tag table 이름 설정
table = 'wind_elec_gen'
# tag name 설정
name = quote(tags_, safe=":/")
# 시간 포멧 설정 
timeformat = 'default'
# Train , validation , test 데이터 셋 설정
# 학습 데이터 시작 시간 설정
start_time = quote('2014-01-01 06:00:00')
# 학습 데이터  끝 시간 설정
end_time = quote('2018-01-01 03:00:00')

In [7]:
# 데이터 로드 함수
# 시간대를 맞추기 위해 3시간으로 resampling
# target 값으로 사용할 통합 발전량 계산후 추가 
def data_load(table, name, start_time, end_time, timeformat):
    
    # 데이터 로드 
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')

    # 같은 시간대 별 데이터로 전환
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()

    # 'TIME' 열을 datetime 형식으로 변환
    df['TIME'] = pd.to_datetime(df['TIME'])

    # time index 설정
    df.set_index('TIME', inplace=True)

    # 3시간 간격으로 reampling 
    df = df.resample('3H').mean()

    # 결측값 제거
    df = df.dropna()

    # 발전량 병합
    # 한경 발전소의 1, 2, 3 발전량을 합침
    df.loc[:, 'hk1_total'] = df.iloc[:, 10:].sum(axis=1)
    
    return df

In [8]:
# 데이터 로드
df = data_load(table, name, start_time, end_time, timeformat)
df

NAME,h_6hrain,h_6hsnow,h_humidity,h_rainprobability,h_raintype,h_seawave,h_skystatus,h_temperature,h_winddirection,h_windspeed,hk1_1,hk1_2,hk1_3,hk1_total
TIME,,,,,,,,,,,,,,
2014-01-01 06:00:00,0.0,0.0,43.0,0.0,0.0,1.5,1.0,12.1,285.0,8.1,1327.916667,-4.666667,1328.900000,2652.150000
2014-01-01 09:00:00,0.0,0.0,50.0,0.0,0.0,1.0,1.0,10.6,285.0,8.0,849.011111,345.961111,813.372222,2008.344444
2014-01-01 12:00:00,0.0,0.0,50.0,0.0,0.0,1.0,1.0,10.2,278.0,8.2,543.388889,522.255556,533.427778,1599.072222
2014-01-01 15:00:00,0.0,0.0,50.0,0.0,0.0,1.0,1.0,8.4,281.0,6.9,489.533333,472.450000,461.377778,1423.361111
2014-01-01 18:00:00,0.0,0.0,50.0,0.0,0.0,1.0,1.0,8.0,298.0,6.4,369.827778,266.494444,342.866667,979.188889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-09 18:00:00,0.0,0.0,65.0,20.0,0.0,0.5,3.0,10.0,220.0,5.1,170.116667,169.338889,144.744444,484.200000
2017-12-09 21:00:00,0.0,0.0,65.0,20.0,0.0,0.5,3.0,10.0,223.0,5.3,267.794444,435.411111,416.900000,1120.105556
2017-12-10 00:00:00,5.0,0.0,70.0,60.0,0.0,1.0,4.0,11.0,254.0,5.8,484.464706,669.894118,702.876471,1857.235294


In [9]:
# train, validation, test 데이터 분리
# valid 2016년 , test 2017년, train 2016년 이전 데이터로 구성 

train = df[df.index.year < 2016]
valid = df[df.index.year == 2016]
test = df[df.index.year == 2017]

## 데이터 전처리

   * 각 데이터 별로 MinMaxScaler 적용
   * 입력 데이터와 targer 데이터 각각 적용 -> test 이후 원래 값으로 변환하기 위함     

In [10]:
# 스케일러 설정
scaler_data = MinMaxScaler()
scaler_target = MinMaxScaler()

# 스케일러 적용
train_ = scaler_data.fit_transform(train.iloc[:,:-1].values)
train_t = scaler_target.fit_transform(train.iloc[:,-1:].values)

valid_ = scaler_data.transform(valid.iloc[:,:-1].values)
valid_t = scaler_target.transform(valid.iloc[:,-1:].values)

test_ = scaler_data.transform(test.iloc[:,:-1].values)
test_t = scaler_target.transform(test.iloc[:,-1:].values)

# 데이터 프레임 설정
train_scaled = pd.DataFrame(train_)
train_scaled['total'] = train_t

valid_scaled = pd.DataFrame(valid_)
valid_scaled['total'] = valid_t

test_scaled = pd.DataFrame(test_)
test_scaled['total'] = test_t

## 윈도우 데이터 셋 설정

시계열 데이터 학습을 위해서는 윈도우 사이즈와 슬라이딩 값을 설정해야함

* window size : 몇개의 시간으로 묶을지에 대한 설정
* step size : window 가 이동하는 시간 간격

In [11]:
# 슬라이딩 윈도우 데이터셋 설정 
class SlidingWindowDataset(Dataset):
    def __init__(self, data, window_size, step_size):
        self.data = data
        self.window_size = window_size
        self.step_size = step_size
        self.windows, self.targets = self._create_windows()
    
    # 슬라이딩 윈도우 설정
    def _create_windows(self):
        windows = []
        targets = []
        for i in range(0, len(self.data) - self.window_size + 1, self.step_size):
            window = self.data.iloc[i:i + self.window_size, :-1].values  # 마지막 컬럼 제외
            target_array = self.data.iloc[i:i + self.window_size, -1].values  # 마지막 컬럼이 target
                
            windows.append(torch.Tensor(window))
            targets.append(torch.Tensor(target_array))  # target을 Tensor로 변환
        return windows, targets
    
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        return self.windows[idx], self.targets[idx]

In [12]:
# 슬라이딩 윈도우 설정
window_size = 8
step_size = 1 

# 데이터 셋 설정 
train_ = SlidingWindowDataset(train_scaled, window_size, step_size)
valid_ = SlidingWindowDataset(valid_scaled, window_size, step_size)
test_ = SlidingWindowDataset(test_scaled, window_size, step_size)

# 데이터 로더 설정
train_dataloader = DataLoader(train_, batch_size=16, shuffle=False)
valid_dataloader = DataLoader(valid_, batch_size=16, shuffle=False)
test_dataloader = DataLoader(test_, batch_size=16, shuffle=False)

In [13]:
print(list(train_dataloader)[0][0].shape)

torch.Size([16, 8, 13])


## 학습 모델 설정 

* LSTM AE 기본 모델 사용

In [14]:
# LSTM Autoencoder 클래스 정의
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(LSTMAutoencoder, self).__init__()
        
        # 인코더 LSTM
        self.encoder_lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.encoder_fc = nn.Linear(hidden_dim, int(hidden_dim/2))
        
        # 디코더 LSTM
        self.decoder_fc = nn.Linear(int(hidden_dim/2), hidden_dim)
        self.decoder_lstm = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True)
        
        # FC Layer 설정
        self.fc1 = nn.Linear(input_dim,6)
        self.fc2 = nn.Linear(6,1)

    def forward(self, x):
        # 인코더 부분
        _, (h, _) = self.encoder_lstm(x)
        latent = self.encoder_fc(h[-1])
        
        # 디코더 부분
        hidden = self.decoder_fc(latent).unsqueeze(0).repeat(x.size(1), 1, 1).permute(1, 0, 2)
        output, _ = self.decoder_lstm(hidden)
        
        # FC Layer
        output = self.fc1(output)
        output = self.fc2(output)
        return output

In [15]:
# 모델 설정 파라미터

# 입력 데이터 컬럼 수
# PCA 한값 
# print(list(train_dataloader)[0][0].shape) 에서 마지막 숫자 
input_dim = 13

# LSMT hidden state 크기
hidden_dim = 6

# layer 수
num_layers = 2

# 학습률 
learning_rate = 0.001

# 모델 초기화
model = LSTMAutoencoder(input_dim, hidden_dim, num_layers).to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 구조 확인
print(model)

LSTMAutoencoder(
  (encoder_lstm): LSTM(13, 6, num_layers=2, batch_first=True)
  (encoder_fc): Linear(in_features=6, out_features=3, bias=True)
  (decoder_fc): Linear(in_features=3, out_features=6, bias=True)
  (decoder_lstm): LSTM(6, 13, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=13, out_features=6, bias=True)
  (fc2): Linear(in_features=6, out_features=1, bias=True)
)


## 모델 학습 설정

* 학습 중 validation 데이터 기준 MSE 값이 제일 낮은 모델을 저장 

In [16]:
train_loss = []
valid_loss = []
total_step = len(train_dataloader)
total_step_ = len(valid_dataloader)
epoch_in = trange(100, desc='training')
best_Loss= np.inf

for epoch in epoch_in:
    model.to(device)
    model.train()
    running_loss = 0.0

    for batch_idx, train_data in enumerate(train_dataloader):

        inputs = train_data[0].to(device).float()
        target = train_data[1].to(device).float()

        optimizer.zero_grad()

        outputs = model(inputs)
        
        loss = criterion(outputs.squeeze(), target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss)}')
    
    # Epoch 마다 validation을 진행해서 가장 좋은 성능을 보이는 모델을 저장 
    with torch.no_grad():
        model.eval()
        running_loss_ = 0.0
        
        for batch_idx, valid_data in enumerate(valid_dataloader):
            
            inputs_ = valid_data[0].to(device).float()
            target_ = valid_data[1].to(device).float()

            outputs_ = model(inputs_)
            
            loss_ = criterion(outputs_.squeeze(), target_)
            
            running_loss_ += loss_.item()
            
        valid_loss.append(running_loss_/total_step_)    

    
    if best_Loss > np.mean(valid_loss):
        best_Loss = np.mean(valid_loss)
        torch.save(model, f'./result/Wind_Elec_Gen_General.pt')
        print('모델 저장')
    epoch_in.set_postfix_str(f"epoch = {epoch}, best_Loss = {best_Loss}")

training:   0%|          | 0/100 [00:00<?, ?it/s]


train loss: 0.10497798808733219
모델 저장


c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1, 8])) that is different to the input size (torch.Size([8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



train loss: 0.07009734050156492
모델 저장

train loss: 0.05669249567678885
모델 저장

train loss: 0.049581676118025504
모델 저장

train loss: 0.045194014430526744
모델 저장

train loss: 0.04221041940257931
모델 저장

train loss: 0.03963830807391609
모델 저장

train loss: 0.03658906462113043
모델 저장

train loss: 0.03393602749237281
모델 저장

train loss: 0.031726155835246676
모델 저장

train loss: 0.029874271303527947
모델 저장

train loss: 0.02830658092610833
모델 저장

train loss: 0.026967094588272726
모델 저장

train loss: 0.025805254146957373
모델 저장

train loss: 0.024788053555734043
모델 저장

train loss: 0.02389106182117813
모델 저장

train loss: 0.02309332792600077
모델 저장

train loss: 0.022380577562833032
모델 저장

train loss: 0.02173865216391254
모델 저장

train loss: 0.021157986733682924
모델 저장

train loss: 0.020629771659825387
모델 저장

train loss: 0.02014707549242262
모델 저장

train loss: 0.019704050160345307
모델 저장

train loss: 0.0192958359218911
모델 저장

train loss: 0.018918352843827545
모델 저장

train loss: 0.018568161532814622
모델 저장

train loss: 

## 모델 테스트

In [17]:
# 베스트 모델 로드
model_ = torch.load(f'./result/Wind_Elec_Gen_General.pt')

In [18]:
# 모델 테스트 
output_test = []
with torch.no_grad():
    model_.eval()
    running_loss_t = 0.0
    
    for batch_idx, test_data in enumerate(test_dataloader):
        
        inputs_t = test_data[0].to(device).float()
        target_t = test_data[1].to(device).float()

        outputs_t = model_(inputs_t)
        
        output_test.append(outputs_t)
        
# 텐서들을 하나로 합치기
combined_tensor = torch.cat(output_test, dim=0)

# window의 마지막 값만 사용하기
last_values = combined_tensor[:, -1, :].cpu()

# 원래 형태의 데이터로 변환
# MinMaxScaler 역변환 
last_values_ = scaler_target.inverse_transform(last_values)

# 결과 데이터 프레임 생성
final_scaled = pd.DataFrame(test_scaled.iloc[7:,-1:].values, columns=['real'])
final_scaled['pred'] = last_values

final = pd.DataFrame(test.iloc[7:, -1:].values, columns=['real'])
final['pred'] = last_values_

In [19]:
# MSE 계산
mse = mean_squared_error(final_scaled['real'].values, final_scaled['pred'].values)
print("Mean Squared Error:", mse)

# R² 스코어 계산
r2 = r2_score(final['real'].values, final['pred'].values)
print("R² Score:", r2)

Mean Squared Error: 0.004653917329540123
R² Score: 0.944772949282327
